# Sorting images by their principal colors

In [48]:
import pandas as pd
from PIL import Image
import math
import colorsys
import os

## Loading the datasets

Sorting results vary depending on the dataset used: with enhanced effect, resized image and numer of clusters used (just 3 clusters vs. using the knee algorithm).

In [49]:
file = "enhanced10p"
df_col = pd.read_csv("ColorCluster_" + file + "_RGB.csv")
df_per = pd.read_csv("ColorCluster_" + file + "_Percentage.csv")

In [50]:
df_col.head(10)

,8bffe092134cac84ac5affab4fb9b738,aecd3acf09c5bf0772ab0a913a4e7eeb,1bd9b38e59eabb23abbcda38a323c8e9,lamp,99fafc0e66a78fdede1bfda2872d7b36,mos,pipe,coc,peach,flowers,...,1f23d405f5268c7e1dd58bcd81a3d358,cx9bc2mc8qx41,36c0c3842ae27d4273b71f631461f44e,1fc604aa185d824765a6b9582df3b6a6,tumblr_ntfry6l8nD1tn6jt3o1_540,knkft95kjh151,ssmFU4nYEHIBA_yMj0q7UZdvsO5kz2Jwb4DmxtC9lXA,tumblr_njz9drX3WF1r38w10o1_540,321faa0baaf1f30cca4af3ae7f0551f6,325vhan02s051
0,"(248, 209, 169)","(237, 112, 122)","(79, 22, 10)","(251, 251, 251)","(244, 172, 215)","(31, 26, 31)","(201, 121, 41)","(240, 243, 244)","(243, 22, 16)","(45, 15, 4)",...,"(39, 13, 5)","(236, 244, 196)","(215, 225, 121)","(149, 211, 19)","(237, 228, 221)","(242, 244, 249)","(9, 23, 32)","(137, 128, 72)","(48, 28, 16)","(4, 23, 0)"
1,"(213, 63, 15)","(102, 8, 13)","(225, 170, 102)","(250, 169, 2)","(28, 24, 13)","(28, 149, 204)","(238, 235, 228)","(28, 10, 5)","(80, 1, 1)","(243, 130, 7)",...,"(244, 228, 174)","(110, 163, 48)","(53, 6, 4)","(62, 7, 98)","(16, 7, 4)","(3, 17, 38)","(223, 239, 244)","(9, 29, 5)","(229, 204, 192)","(204, 252, 100)"
2,"(40, 6, 3)","(253, 221, 229)","(252, 246, 229)","(178, 66, 5)","(179, 87, 112)","(219, 202, 171)","(35, 15, 24)","(165, 149, 143)","(255, 143, 107)","(234, 227, 191)",...,"(231, 139, 31)","(7, 49, 1)","(165, 32, 52)","(185, 46, 239)","(146, 112, 88)","(127, 142, 167)","(106, 136, 151)","(228, 237, 222)","(168, 139, 125)","(106, 186, 14)"


In [51]:
df_per.head(10)

,8bffe092134cac84ac5affab4fb9b738,aecd3acf09c5bf0772ab0a913a4e7eeb,1bd9b38e59eabb23abbcda38a323c8e9,lamp,99fafc0e66a78fdede1bfda2872d7b36,mos,pipe,coc,peach,flowers,...,1f23d405f5268c7e1dd58bcd81a3d358,cx9bc2mc8qx41,36c0c3842ae27d4273b71f631461f44e,1fc604aa185d824765a6b9582df3b6a6,tumblr_ntfry6l8nD1tn6jt3o1_540,knkft95kjh151,ssmFU4nYEHIBA_yMj0q7UZdvsO5kz2Jwb4DmxtC9lXA,tumblr_njz9drX3WF1r38w10o1_540,321faa0baaf1f30cca4af3ae7f0551f6,325vhan02s051
0,17.0,33.0,16.0,50.0,44.0,53.0,28.0,66.0,64.0,27.0,...,41.0,30.0,30.0,36.0,31.0,34.0,35.0,23.0,9.0,28.0
1,52.0,18.0,28.0,38.0,29.0,27.0,48.0,18.0,14.0,58.0,...,17.0,22.0,43.0,38.0,49.0,51.0,35.0,56.0,44.0,44.0
2,31.0,50.0,57.0,12.0,27.0,21.0,24.0,16.0,22.0,15.0,...,42.0,48.0,28.0,25.0,20.0,15.0,30.0,21.0,47.0,28.0


## Extract the two maximum values per column and their percentages

In [52]:
first_max = []
second_max = []
imgs_name = []
percentage = []
percentage2 = []
for i in df_col.keys():
    imgs_name = imgs_name + [i]
    maxs = df_per[i].nlargest(2).keys()
    first_max = first_max + [df_col[i][maxs[0]]]
    percentage = percentage + [df_per[i][maxs[0]]]
    second_max = second_max + [df_col[i][maxs[1]]]
    percentage2 = percentage2 + [df_per[i][maxs[1]]]

In [53]:
from ast import literal_eval as make_tuple

first_max = list(map(make_tuple, first_max))
second_max = list(map(make_tuple, second_max))

## Color sorting and color stripe image

Once determined the principal color(s) for each picture, now it's the time to sort them. There are many ways to do that, but I will focus in the RGB ans HSV color spaces. 

The sorting method depends mainly on the dataset, and to choose the best one it's useful to look at the color stripes for each method generated by the *color_stripe* function.

In [54]:
from color_analysis import color_stripe, SortingColors

### Luminosity sorting

Sorts by the percieved luminosity of the colors

In [55]:
def lum (r,g,b):
    return math.sqrt( 0.299*(r**2) + 0.587*(g**2) + 0.114*(b**2) )

In [56]:
lum_t = [lum(*i) for i in first_max]
LUM = SortingColors((imgs_name, first_max, lum_t), 2)
color_stripe([20,100], LUM.sorted_RGBvalues, 'ColorStripe_LUM')
print(max(lum_t), min(lum_t))

251.5291474163581 0.9999999999999999


Also, the luminosity can help determine the principal color. 

With Knn we can extract a number of color clusters from an image (maybe a fixed number like 3 clusters for example, or maybe a variable number using the knee algorithm), but this algorithm has a downside: if an image is composed by various objects, the shadows between them will be percieved as a color cluster in itself and in some cases it may be even considered the principal color when in reality it's just noise.

Using the luminosity we can determine if the principal color is really the principal color of the image or noise product of the shadows. Implementing an inferior threshold for the luminosity of the "principal color", combined with a threshold for the percentage of pixels with that particular color, we can fix this problem replacing the principal color cluster with the second principal color cluster in the images with noise, while undisturbing the images which are truly darker.

In [57]:
max_withlum = []

for i in range(0, len(first_max)):
    
    if (lum_t[i]<100 and abs(percentage[i]-percentage2[i])<10):
        max_withlum = max_withlum + [second_max[i]]
    
    elif (lum_t[i]>230 and abs(percentage[i]-percentage2[i])<10):
        max_withlum = max_withlum + [second_max[i]]
        
    else:
        max_withlum = max_withlum + [first_max[i]]

### RGB sorting

The most simple way of sorting. First, it sorts the first component (R: red), then the second (G: green) and finally the third (B: blue) of the principal color of the image.

In [58]:
RGB = SortingColors((imgs_name, first_max), 1)
color_stripe([20,100], RGB.sorted_RGBvalues, 'ColorStripe_RGB')
RGB.sorted[0:10]

[('75bdbb88471277b3e4a4d2fd7dd510e4', (255, 251, 245)),
 ('8881e2f9bc476775f8d5b2df27604fe0', (255, 239, 162)),
 ('pf', (255, 216, 2)),
 ('373cc3217e11b7059d0ec44b1480f617', (255, 188, 191)),
 ('gf', (255, 7, 66)),
 ('0b813c3ff57ebc81d55e8ccd1dc21788', (254, 233, 1)),
 ('bee', (254, 231, 63)),
 ('flo', (254, 212, 234)),
 ('2df432f958ac518069cfd3a592e3b541', (254, 88, 1)),
 ('68788d484729dc0297da343a0afe835e', (253, 224, 7))]

### HSV sorting

The HSV color space represents the similarity between colors better than the RGB space. It uses hue, saturation and value as parameters.

In [59]:
def hsv(x):     
    x = [i/255 for i in x]
    return(colorsys.rgb_to_hsv(*x))

In [60]:
hsv_t = [hsv(i) for i in first_max]
HSV = SortingColors((imgs_name, first_max, hsv_t), 2)
color_stripe([20,100], HSV.sorted_RGBvalues, 'ColorStripe_HSV')
HSV.sorted[0:10]

[('9faa0909c6e6f741c42c27abbe32f861',
  (251, 2, 3),
  (0.999330655957162, 0.9920318725099602, 0.984313725490196)),
 ('aa89e981b6a63fa8a1fa253d68c7a2a3',
  (75, 5, 6),
  (0.9976190476190476, 0.9333333333333333, 0.29411764705882354)),
 ('peach2',
  (232, 28, 31),
  (0.9975490196078431, 0.8793103448275863, 0.9098039215686274)),
 ('638d3fc4ecd8471815769a763fd436c4',
  (231, 3, 11),
  (0.9941520467836258, 0.987012987012987, 0.9058823529411765)),
 ('373cc3217e11b7059d0ec44b1480f617',
  (255, 188, 191),
  (0.9925373134328359, 0.26274509803921564, 1.0)),
 ('0a8616ebe57d001669e7c22bafdf3b22',
  (250, 59, 77),
  (0.9842931937172775, 0.764, 0.9803921568627451)),
 ('oct',
  (253, 220, 225),
  (0.9747474747474748, 0.13043478260869562, 0.9921568627450981)),
 ('ic2',
  (60, 4, 13),
  (0.9732142857142857, 0.9333333333333333, 0.23529411764705882)),
 ('b64b285fd18e272b85043332f06fb499',
  (243, 138, 156),
  (0.9714285714285714, 0.4320987654320988, 0.9529411764705882)),
 ('a2cbb62b458a8eb342231de5b82501

### HLS sorting

HLS is similar to HSV, but instead the color space is organized by hue, saturation and lightness

In [61]:
def hls(x):
    x = [i/255 for i in x]
    return(colorsys.rgb_to_hls(*x))

In [62]:
hls_t = [hls(i) for i in first_max]
HLS = SortingColors((imgs_name, first_max, hls_t), 2)
color_stripe([20,100], HLS.sorted_RGBvalues, 'ColorStripe_HLS')
HLS.sorted[0:10]

[('9faa0909c6e6f741c42c27abbe32f861',
  (251, 2, 3),
  (0.999330655957162, 0.496078431372549, 0.9841897233201582)),
 ('aa89e981b6a63fa8a1fa253d68c7a2a3',
  (75, 5, 6),
  (0.9976190476190476, 0.1568627450980392, 0.8750000000000001)),
 ('peach2', (232, 28, 31), (0.9975490196078431, 0.5098039215686274, 0.816)),
 ('638d3fc4ecd8471815769a763fd436c4',
  (231, 3, 11),
  (0.9941520467836258, 0.4588235294117647, 0.9743589743589745)),
 ('373cc3217e11b7059d0ec44b1480f617',
  (255, 188, 191),
  (0.9925373134328359, 0.8686274509803922, 1.0)),
 ('0a8616ebe57d001669e7c22bafdf3b22',
  (250, 59, 77),
  (0.9842931937172775, 0.6058823529411764, 0.9502487562189056)),
 ('oct',
  (253, 220, 225),
  (0.9747474747474748, 0.9274509803921569, 0.8918918918918922)),
 ('ic2', (60, 4, 13), (0.9732142857142857, 0.12549019607843137, 0.875)),
 ('b64b285fd18e272b85043332f06fb499',
  (243, 138, 156),
  (0.9714285714285714, 0.7470588235294118, 0.8139534883720928)),
 ('a2cbb62b458a8eb342231de5b825015c',
  (222, 126, 143),

### Step sorting

The first one is based on the HSV color space, but also sorts by luminosity, generating a number of color segments. The ideal number of segments varies from dataset to dataset. And to add continuity to the segments, it inverts the luminosity every two segments.

In [63]:
def step_lum (r,g,b, repetitions=1):
    
    lum = math.sqrt( 0.299*(r**2) + 0.587*(g**2) + 0.114*(b**2) )
    h, s, v = colorsys.rgb_to_hsv(r/255,g/255,b/255)
    
    h2 = int(h * repetitions)
    lum2 = int(lum * repetitions)
    v2 = int(v * repetitions)
    
    if h2 % 2 == 1:
        v2 = repetitions - v2
        lum = repetitions - lum
        
    return (h2, lum, v2)


In [64]:
for number_steps in range(0,20):
    steplum_t = [step_lum(*i, number_steps) for i in max_withlum]
    STEP_LUM = SortingColors((imgs_name, max_withlum, steplum_t), 2)
    color_stripe([20,100], STEP_LUM.sorted_RGBvalues, 'ColorStripe_STEPLUM' + str(number_steps))

## Plot the sorted images for a better perception of the images' order and save the order in a csv file

First, select the best sorting option for the dataset. In this case, I think the best one is the step sorting with luminosity fix using nine steps, and enhanced and resized by 10% pictures.

In [69]:
final_t = [step_lum(*i, 9) for i in max_withlum]
FINAL = SortingColors((imgs_name, first_max, final_t), 2)

In [66]:
from color_analysis import collage

In [70]:
outfilename = 'resized10p_enhanced_9steps_lumfix'
collage(FINAL.sorted_names, 400, 4, "Imgs_done", "collage_" + outfilename)

## Save the best result

In [71]:
final_df = pd.DataFrame(list(zip(FINAL.sorted_names, FINAL.sorted_RGBvalues)), columns = ['sorted_names', 'sorted_RGBcolors'])
final_df.to_csv('collage/resized10p_enhanced_9steps_lumfix.csv', index = False)